In [1]:
from dask_jobqueue import SLURMCluster # pip 
from dask.distributed import Client, progress 
import dask

import re
import matplotlib as mpl
# Define Agg as Backend for matplotlib when no X server is running
mpl.use('Agg')
import socket
import os
import importlib 

import inspect
import timeit

In [2]:

def squeue_user(username = "$USER"):
    _squeue_user = os.system("squeue --user="+str(username))
    return _squeue_user 

with os.popen("pwd ") as f:
    _pwd = f.readline()
pwd = re.split(r'[\n]', _pwd)[0]

In [3]:

extra_args=[
    "--error="+str(pwd)+"/slurm/logs/dask-worker-%j.err",
    "--output="+str(pwd)+"/slurm/output/dask-worker-%j.out"
]
cluster = SLURMCluster(
    name='dask-cluster', 
    cores=256,    
    memory="500 GB", 
    project="bb1153",
    queue= "compute", 
    walltime='04:30:50',
)
client = Client(cluster)
print(cluster.job_script())

/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=256
#SBATCH --mem=466G
#SBATCH -t 04:30:50

/home/b/b382267/mambaforge/envs/tropical-rainfall/bin/python -m distributed.cli.dask_worker tcp://136.172.124.7:36637 --nthreads 16 --nworkers 16 --memory-limit 29.10GiB --name dummy-name --nanny --death-timeout 60



/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44161 instead
  warnings.warn(
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


In [4]:
cluster.scale(jobs=1)

In [10]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4509090   compute dask-wor  b382267 PD       0:00      1 (Priority)
           4509083   compute dask-wor  b382267  R       6:17      1 l40301


0

In [7]:
os.system("scancel  4265508") 

0

In [21]:
while True:
    try:
        importlib.reload(src.shared_func)
        from  src.shared_func import time_interpreter,  animation
        break
    except NameError:
        import src.shared_func
        from  src.shared_func import time_interpreter,  animation
        break

In [16]:
import sys
sys.path.append('..')

In [18]:
while True:
    try:
        importlib.reload(src.tr_pr_mod)
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break
    except NameError:
        import src.tr_pr_mod
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break

In [38]:
diag = TR_PR_Diagnostic()

diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 2000*10**(-6)/diag.num_of_bins

last_edge = diag.first_edge  + diag.num_of_bins*diag.width_of_bin

In [39]:
import aqua

In [41]:
from aqua import Reader
from aqua.reader import catalogue

In [50]:
diagname  = 'tr_pr'
machine   = 'levante'

In [54]:
catalogue()

SystemExit: Cannot find the basic configuration file!

/home/b/b382267/mambaforge/envs/aqua/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
reader = Reader(model="ICON", exp="ngc2009", source="atm_2d_ml_R02B09")
ICON_2009 = reader.retrieve()

In [ ]:
ICON_2009

In [13]:
import xarray as xr 
import matplotlib.pyplot as plt
import numpy as np 

In [14]:
ICON_2009_small = ICON_2009['pr'][0:3,:]
ICON_2009_small = ICON_2009_small.compute()

In [102]:
diag.time_interpreter_b(ICON_2009)

diag.time_interpreter_b(ICON_2009_small) 

'30m'

### Numpy.package (Numpy.digitize + Numpy.count_nonzero)

In [59]:
%timeit -r 2 -n 1 -o diag.hist_np_digitize(ICON_2009_small,  preprocess = False)

6.94 s ± 259 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 6.94 s ± 259 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [60]:
result = _
#vars(result)
time_np_digitize, time_np_digitize_stdev =  result.average, result.stdev 

In [103]:
hist_digitize = diag.hist_np_digitize(ICON_2009_small,  preprocess = False)

(<xarray.DataArray (bin: 15)>
 array([62657269,   119221,    43543,    21621,    13658,     9689,
            7086,     5303,     4263,     3534,     2761,     2251,
            2086,     1822,     1621])
 Coordinates:
   * bin      (bin) float64 0.0 0.0001333 0.0002667 ... 0.0016 0.001733 0.001867,
 62914560,
 6.729786157608032)

### fast_histogram package (fast_histogram.histogram1d)

In [62]:
%timeit -r 2 -n 1 -o diag.hist1d_fast(ICON_2009_small,  preprocess = False)

257 ms ± 2.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 257 ms ± 2.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [63]:
result = _
time_fast, time_fast_stdev =  result.average, result.stdev 

In [104]:
hist_fast  = diag.hist1d_fast(ICON_2009_small,  preprocess = False)

### Numpy package (Numpy.histogram)

In [65]:
%timeit -r 2 -n 1 -o diag.hist1d_np(ICON_2009_small,  preprocess = False)

450 ms ± 40.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 450 ms ± 40.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [66]:
result = _
time_np, time_np_stdev =  result.average, result.stdev 

In [105]:
hist_np = diag.hist1d_np(ICON_2009_small,  preprocess = False)

In [67]:
ICON_2009_small_prep = ICON_2009_small.stack(total=['time', 'cell'])

###  matplotlib.pyplot package (matplotlib.pyplot.hist)

In [69]:
%timeit -r 2 -n 1 -o diag.hist1d_pyplot(ICON_2009_small_prep,  preprocess = False)

1.85 s ± 82.2 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 1.85 s ± 82.2 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [70]:
result = _
time_pyplt, time_pyplt_stdev =  result.average, result.stdev 

In [106]:
hist_pyplt = diag.hist1d_pyplot(ICON_2009_small_prep,  preprocess = False)

## Dask packages

In [71]:
import dask.array as da

In [72]:
ICON_2009_small_prep_dask = da.from_array(ICON_2009_small_prep)

### dask_histogram.boost package (dask_histogram.boost.Histogram)

In [73]:
%timeit -r 2 -n 1 -o diag.dask_boost(ICON_2009_small_prep_dask,  preprocess = False)

dask.array<to-dask-array, shape=(15,), dtype=float64, chunksize=(15,), chunktype=numpy.ndarray> [dask.array<array, shape=(16,), dtype=float64, chunksize=(16,), chunktype=numpy.ndarray>]
dask.array<to-dask-array, shape=(15,), dtype=float64, chunksize=(15,), chunktype=numpy.ndarray> [dask.array<array, shape=(16,), dtype=float64, chunksize=(16,), chunktype=numpy.ndarray>]
6.24 s ± 528 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 6.24 s ± 528 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [74]:
result = _
time_boost, time_boost_stdev =  result.average, result.stdev 

In [107]:
hist_boost = diag.dask_boost(ICON_2009_small_prep_dask,  preprocess = False)

dask.array<to-dask-array, shape=(15,), dtype=float64, chunksize=(15,), chunktype=numpy.ndarray> [dask.array<array, shape=(16,), dtype=float64, chunksize=(16,), chunktype=numpy.ndarray>]


### dask_histogram package  (dask_histogram.factory)

In [75]:
%timeit -r 2 -n 1 -o diag.dask_factory(ICON_2009_small_prep_dask,  preprocess = False)

1.54 s ± 315 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 1.54 s ± 315 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [76]:
result = _
time_factory, time_factory_stdev =  result.average, result.stdev 

In [108]:
hist_factory = diag.dask_factory(ICON_2009_small_prep_dask,  preprocess = False)

In [ ]:
# Check the scaling 

# lat + lon coordinates

In [77]:
reader = Reader(model="ICON", exp="ngc2009", source="atm_2d_ml_R02B09", regrid="r200")

ICON_2009_reg = reader.retrieve()

No fixes defined for model  ICON


In [78]:
ICON_2009_reg_small = ICON_2009_reg['pr'][0:3,:]
ICON_2009_reg_small = ICON_2009_reg_small.compute()

In [79]:
diag.time_interpreter_b(ICON_2009_reg_small) 

'30m'

### Numpy.package (Numpy.digitize + Numpy.count_nonzero)

In [80]:
%timeit -r 2 -n 1 -o diag.hist_np_digitize(ICON_2009_reg_small,  preprocess = False)

6.56 s ± 180 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 6.56 s ± 180 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [81]:
result = _
time_digitize_reg, time_digitize_reg_stdev =  result.average, result.stdev 

In [109]:
hist_digitize_reg = diag.hist_np_digitize(ICON_2009_reg_small,  preprocess = False)

### fast_histogram package (fast_histogram.histogram1d)

In [82]:
%timeit -r 2 -n 1 -o  diag.hist1d_fast(ICON_2009_reg_small,  preprocess = False)

251 ms ± 484 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 251 ms ± 484 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [83]:
result = _
time_fast_reg, time_fast_reg_stdev =  result.average, result.stdev 

In [110]:
hist_fast_reg = diag.hist1d_fast(ICON_2009_reg_small,  preprocess = False)

### Numpy package (Numpy.histogram)

In [87]:
%timeit -r 2 -n 1 -o  diag.hist1d_np(ICON_2009_reg_small,  preprocess = False)

430 ms ± 485 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 430 ms ± 485 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [88]:
result = _
time_np_reg, time_np_reg_stdev =  result.average, result.stdev 

In [111]:
hist_np_reg = diag.hist1d_np(ICON_2009_reg_small,  preprocess = False)

In [89]:
ICON_2009_reg_small_prep = ICON_2009_reg_small.stack(total=['time', 'cell'])

###  matplotlib.pyplot package (matplotlib.pyplot.hist)

In [90]:
%timeit -r 2 -n 1 -o  diag.hist1d_pyplot(ICON_2009_reg_small_prep,  preprocess = False)

1.77 s ± 3.76 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 1.77 s ± 3.76 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [91]:
result = _
time_pyplt_reg, time_pyplt_reg_stdev =  result.average, result.stdev 

In [112]:
hist_pyplt_reg = diag.hist1d_pyplot(ICON_2009_reg_small_prep,  preprocess = False)

In [92]:
ICON_2009_reg_small_prep_dask = da.from_array(ICON_2009_reg_small_prep)

### dask_histogram package  (dask_histogram.factory)

In [93]:
%timeit -r 2 -n 1 -o  diag.dask_boost(ICON_2009_reg_small_prep_dask,  preprocess = False)

dask.array<to-dask-array, shape=(15,), dtype=float64, chunksize=(15,), chunktype=numpy.ndarray> [dask.array<array, shape=(16,), dtype=float64, chunksize=(16,), chunktype=numpy.ndarray>]
dask.array<to-dask-array, shape=(15,), dtype=float64, chunksize=(15,), chunktype=numpy.ndarray> [dask.array<array, shape=(16,), dtype=float64, chunksize=(16,), chunktype=numpy.ndarray>]
5.32 s ± 105 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 5.32 s ± 105 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [94]:
result = _
time_boost_reg, time_boost_reg_stdev =  result.average, result.stdev 

In [113]:
hist_boost_reg = diag.dask_boost(ICON_2009_reg_small_prep_dask,  preprocess = False)

dask.array<to-dask-array, shape=(15,), dtype=float64, chunksize=(15,), chunktype=numpy.ndarray> [dask.array<array, shape=(16,), dtype=float64, chunksize=(16,), chunktype=numpy.ndarray>]


### dask_histogram.boost package (dask_histogram.boost.Histogram)

In [95]:
%timeit -r 2 -n 1 -o  diag.dask_factory(ICON_2009_reg_small_prep_dask,  preprocess = False)

1.21 s ± 3.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 1.21 s ± 3.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)>

In [96]:
result = _
time_factory_reg, time_factory_reg_stdev =  result.average, result.stdev 

In [114]:
hist_factory_reg = diag.dask_factory(ICON_2009_reg_small_prep_dask,  preprocess = False)

In [99]:
time_set1 = [ time_boost_reg, time_factory_reg, time_fast_reg,  time_np_reg, time_pyplt_reg,  time_digitize_reg]

time_set2 = [ time_boost, time_factory, time_fast,  time_np, time_pyplt,  time_np_digitize ]

fig = plt.figure(figsize=(12,6))

barWidth = 0.2
br1 = np.arange(len(time_set1))
br2 = [x + barWidth for x in br1]

plt.bar(br1, time_set1, color ='red',  width = barWidth,
        edgecolor ='grey', label = "ICON, regrided") #'aqua, 3.10.9')

plt.bar(br2, time_set2, color ='green',  width = barWidth,
        edgecolor ='grey', label = "ICON") #'aqua, 3.10.9')
#plt.bar(br1, time_set1_3_9_16, color ='red', alpha =0.6, width = barWidth, bottom = time_set1_3_8_16,
#        edgecolor ='grey', label ='CMIP, 3.9.16')
 
# Adding Xticks
plt.xlabel('Methods', fontweight ='bold', fontsize = 15)
plt.ylabel('Calculation time per element', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(time_set1))],
        ['boost', 'factory', 'fast', 'np', 'pyplt', 'digitize'], fontsize=14)  #, 'left'
plt.xticks(fontsize=14)         



plt.legend(fontsize=14)


plt.savefig("./figures/Calc_time_timeit.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')


In [168]:
def check_equality():
    result = _
    if np.count_nonzero(result == True) == result.size:
        return 0
    else:
        return 1

In [ ]:
hist_digitize_reg == hist_digitize

In [ ]:
check_equality()

In [177]:
hist_fast_reg == hist_fast


<xarray.DataArray (bin: 15)>
array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])
Coordinates:
  * bin      (bin) float64 0.0 0.0001333 0.0002667 ... 0.0016 0.001733 0.001867

In [178]:
check_equality()

0

In [172]:
hist_np_reg == hist_np

<xarray.DataArray (bin: 15)>
array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])
Coordinates:
  * bin      (bin) float32 0.0 0.0001333 0.0002667 ... 0.0016 0.001733 0.001867

In [173]:
check_equality()

0

In [34]:

fig = plt.figure()
ax = fig.add_subplot(111)

_pdf = True

diag.hist_plot(hist_fast_reg,           pdf=_pdf,   color = 'tab:red',     label='fast')
diag.hist_plot(hist_np_reg,             pdf=_pdf,   color = 'tab:green',   label='np')
diag.hist_plot(hist_pyplt_reg,          pdf=_pdf,   color = 'tab:blue',    label='pyplt')
diag.hist_plot(hist_digitize_reg,       pdf=_pdf,   color = 'tab:blue',    label='pyplt')

diag.hist_plot(hist_fast,           pdf=_pdf,   color = 'tab:red',     label='fast')
diag.hist_plot(hist_np,             pdf=_pdf,   color = 'tab:green',   label='np')
diag.hist_plot(hist_pyplt,          pdf=_pdf,   color = 'tab:blue',    label='pyplt')
diag.hist_plot(hist_digitize,       pdf=_pdf,   color = 'tab:blue',    label='pyplt')



ax.grid()

plt.savefig("./figures/pdf_hists_reg.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

hist_plot function in the process 

hist_plot function in the process 

hist_plot function in the process 

hist_plot function in the process 

hist_plot function in the process 

hist_plot function in the process 

hist_plot function in the process 

hist_plot function in the process 



In [173]:

fig = plt.figure()
ax = fig.add_subplot(111)

_pdf = True

diag.hist_plot(hist_boost,          pdf=_pdf,   color = 'tab:purple',  label='right')
diag.hist_plot(hist_factory,        pdf=_pdf,   color = 'tab:grey',    label='left')
diag.hist_plot(hist_fast,           pdf=_pdf,   color = 'tab:red',     label='fast')
diag.hist_plot(hist_np,             pdf=_pdf,   color = 'tab:green',   label='np')
diag.hist_plot(hist_pyplt,          pdf=_pdf,   color = 'tab:blue',    label='pyplt')
diag.hist_plot(hist_digitize,       pdf=_pdf,   color = 'tab:blue',    label='pyplt')



ax.grid()

plt.savefig("./figures/pdf_hists_2.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

hist_plot function in the process 

hist_plot function in the process 

hist_plot function in the process 

hist_plot function in the process 

hist_plot function in the process 

hist_plot function in the process 



In [189]:

fig = plt.figure()
ax = fig.add_subplot(111)

_pdf = True

diag.hist_plot(CMIP_hist_boost,     pdf=_pdf,   color = 'tab:purple',  label='right')
diag.hist_plot(CMIP_hist_fact,      pdf=_pdf,   color = 'tab:grey',    label='left')
diag.hist_plot(CMIP_hist_fast,      pdf=_pdf,   color = 'tab:red',     label='fast')
diag.hist_plot(CMIP_hist_np,        pdf=_pdf,   color = 'tab:green',   label='np')
diag.hist_plot(CMIP_hist_pyplt,     pdf=_pdf,   color = 'tab:blue',    label='pyplt')
diag.hist_plot(CMIP_hist_digitize,  pdf=_pdf,   color = 'tab:blue',    label='pyplt')



ax.grid()

plt.savefig("./figures/pdf_hists.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')



hist_plot function in the process 

pdf
hist_plot function in the process 

pdf
hist_plot function in the process 

pdf
hist_plot function in the process 

pdf
hist_plot function in the process 

pdf
hist_plot function in the process 

pdf


In [ ]:
# timeit 

In [36]:
time_set1 = [  del_time_fast_reg,  del_time_np_reg, del_time_pyplt_reg,  del_time_digitize_reg ]

time_set2 = [  del_time_fast,  del_time_np, del_time_pyplt,  del_time_digitize ]

Per_element = True

if Per_element == True:
    time_set1 = [time_set1[i]/size_reg for i in range(len(time_set1))] 

if Per_element == True:
    time_set2 = [time_set2[i]/size for i in range(len(time_set2))] 

fig = plt.figure(figsize=(12,6))

barWidth = 0.2
br1 = np.arange(len(time_set1))
br2 = [x + barWidth for x in br1]

plt.bar(br1, time_set1, color ='red',  width = barWidth,
        edgecolor ='grey', label = "ICON, reg") #'aqua, 3.10.9')

plt.bar(br2, time_set2, color ='green',  width = barWidth,
        edgecolor ='grey', label = "ICON") #'aqua, 3.10.9')
#plt.bar(br1, time_set1_3_9_16, color ='red', alpha =0.6, width = barWidth, bottom = time_set1_3_8_16,
#        edgecolor ='grey', label ='CMIP, 3.9.16')
 
# Adding Xticks
plt.xlabel('Methods', fontweight ='bold', fontsize = 15)
plt.ylabel('Calculation time per element', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(time_set1))],
        ['fast', 'np', 'pyplt', 'digitize'], fontsize=14)  #, 'left'
plt.xticks(fontsize=14)         



plt.legend(fontsize=14)


plt.savefig("./figures/Calc_time_sameds.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')


